In [2]:
import pandas as pd

In [3]:
#import csv
df_rx = pd.read_csv('rx_data.csv')
df_prod = pd.read_csv('product_data.csv')
df_prod = pd.read_csv('Product_Export_20170506_204231.csv')

In [4]:
#format dataframes before merging
df_rx['rxid'] = df_rx['Rx Number']
df_rx.drop_duplicates('rxid',keep='last',inplace=True)
df_rx['date'] = pd.to_datetime(df_rx['Date Written'], format='%m%d%Y')
# df_rx['ndc'] = pd.np.abs(df_rx['Dispensed NDC #'].astype(int))
df_rx['ndc'] = df_rx['Dispensed NDC #']
df_rx['qty'] = df_rx['Refill Qty'].astype(int)
df_rx['days_supply'] = pd.to_numeric(df_rx['Days Supply'],errors='coerce').astype(int)


df_prod['gcsn'] = df_prod['dg-generic-code-seq-nbr']
df_prod['pkg_size'] = df_prod['dg-pkg-size-units']
df_prod['units'] = df_prod['dg-dispensing-units']
df_prod['desc'] = df_prod['dg-description']
df_prod['gen_desc'] = df_prod['dg-description-generic']
df_prod['ppc'] = df_prod['dg-preferred-product-code'].fillna(0)
df_prod['ndc'] = df_prod['dg-ndc']
df_prod['qoh'] = pd.to_numeric(df_prod['dg-qty-on-hand'],errors='coerce').fillna(0).astype(int)
df_prod['reorder_point'] = df_prod["dg-reorder-point"]
df_prod['reorder_qoh_desired'] = df_prod["dg-qty-on-hand-desired"]
df_prod['reorder_qty'] = df_prod["dg-qty-reorder"]


#trim product DF and merge with rx DF
df_all = df_rx.merge(df_prod,how='left',on='ndc')

#trim to relevent columns
df_final = df_all.loc[:,['rxid','date','desc','ndc','qty','days_supply','gcsn','pkg_size','units','gen_desc','ppc','qoh']]
df_final.loc[:,'qty_per_day'] = df_final.loc[:,'qty']/df_final.loc[:,'days_supply']
df_final.dropna(subset=['gcsn','ndc'],inplace=True)

# Perform aggregation

In [5]:
def agg_sum(df,cols,val_col='qty',func=sum):
    return df.groupby(cols + ['date'])[val_col].apply(func).unstack(level=cols).sort_index().sort_index(axis=1)

## First question:

For each item on formulary, i.e. each gcsn with a ppc set to 1 for SOME NDC, what is the:
1. Current preffered product info
    - package size (pkg_size)
    - unit (units)
2. Information on that product/gcsn: 
    - Number of Rx dispensed
    - total units dispense
3. PER RX statistic:
    - max qty
    - mean qty
    - quartiles (25%, 50%, 75%)
    - max qty/days_supply
    - mean qty/days_supply
3. PER CLINIC statistics:
    - Number of Rx dispensed
    - max qty
    - mean qty
    - quartiles (25%, 50%, 75%)

## 1. Prefferred Product info

In [6]:
is_ppc_reorder  = ~(pd.isnull(df_prod["dg-qty-on-hand-desired"]) & pd.isnull(df_prod["dg-qty-reorder"])) & (df_prod["ppc"] == 1)
df_ppc = df_prod.loc[is_ppc_reorder]
df_ppc = df_ppc[['ndc','gcsn','desc','pkg_size','units','reorder_point','reorder_qty','reorder_qoh_desired']]
df_ppc.set_index('gcsn', inplace=True)
df_ppc.columns = pd.MultiIndex.from_tuples([('pp_info',col) for col in df_ppc.columns])
print df_ppc.shape
df_ppc.head()

(283, 7)


pp_info                                                 \
                   ndc                            desc pkg_size units   
gcsn                                                                    
NaN       9.321300e+07  **NITROFURANTOIN MCR 50 MG CAT      100    CA   
NaN       6.846202e+10  **OMEPRAZOLE DR 40 MG CAPSULEG     1000    CA   
 16408.0  6.144201e+10     ACYCLOVIR 400 MG TABLET CAR      500    TA   
 63901.0  4.928104e+10         ADACEL TDAP SYRINGE ATP        1    ML   
 43366.0  1.730695e+08        ADVAIR 100-50 DISKUS GSK       60    EA   

                                                        
         reorder_point reorder_qty reorder_qoh_desired  
gcsn                                                    
NaN                NaN       100.0                 NaN  
NaN                NaN      3000.0                 NaN  
 16408.0         500.0         NaN              2000.0  
 63901.0           1.0         NaN                 1.0  
 43366.0         300.0         NaN               300.0

## 2. Information on GCSN

In [14]:
cnt = df_final.groupby('gcsn').date.count()
cnt.name = 'rx_count'
cnt=cnt.to_frame()
total = df_final.groupby('gcsn').qty.sum()
total.name = 'tot_units'
total=total.to_frame()
qoh = df_prod.groupby('gcsn').qoh.sum()
qoh.name = 'qoh'
qoh=qoh.to_frame()
df_gcsn_agg = cnt.join(total,how='outer')
df_gcsn_agg = df_gcsn_agg.join(qoh,how='outer').fillna(0).sort_values('rx_count')

df_gcsn_agg.columns = pd.MultiIndex.from_tuples([('gcsn_totals',col) for col in df_gcsn_agg.columns])

In [15]:
df_final[df_final['gcsn'] == 1740]

,rxid,date,desc,ndc,qty,days_supply,gcsn,pkg_size,units,gen_desc,ppc,qoh,qty_per_day
44,6000071,2016-10-15,PAP-HUMULIN N 100 UNITS/ML VIA,3000000048,80,133,1740.0,10.0,ML,NaN,2.0,840.0,0.601504
93,6000139,2016-10-18,NOVOLIN N 100 UNITS/ML VIALNOV,169183411,30,60,1740.0,10.0,ML,INSULIN NPH HUMAN ISOPHANE,0.0,10.0,0.500000
169,6000215,2016-10-25,HUMULIN N 100 UNITS/ML VIALLIL,2831501,10,20,1740.0,10.0,ML,INSULIN NPH HUMAN ISOPHANE,1.0,290.0,0.500000
249,6000296,2016-10-29,HUMULIN N 100 UNITS/ML VIALLIL,2831501,30,50,1740.0,10.0,ML,INSULIN NPH HUMAN ISOPHANE,1.0,290.0,0.600000
332,6000377,2016-11-01,NOVOLIN N 100 UNITS/ML VIALNOV,169183411,20,30,1740.0,10.0,ML,INSULIN NPH HUMAN ISOPHANE,0.0,10.0,0.666667
651,6000697,2016-11-12,NOVOLIN N 100 UNITS/ML VIALNOV,169183411,9,90,1740.0,10.0,ML,INSULIN NPH HUMAN ISOPHANE,0.0,10.0,0.100000
867,6000913,2016-11-22,HUMULIN N 100 UNITS/ML VIALLIL,2831501,3,115,1740.0,10.0,ML,INSULIN NPH HUMAN ISOPHANE,1.0,290.0,0.026087
1132,6001177,2016-12-06,HUMULIN N 100 UNITS/ML VIALLIL,2831501,7,120,1740.0,10.0,ML,INSULIN NPH HUMAN ISOPHANE,1.0,290.0,0.058333
1292,6001337,2016-12-13,PAP-HUMULIN N 100 UNITS/ML VIA,3000000048,60,90,1740.0,10.0,ML,NaN,2.0,840.0,0.666667
1300,6001346,2016-12-13,PAP-NOVOLIN N 100 UNITS/ML VIA,3000000078,120,171,1740.0,10.0,ML,NaN,2.0,240.0,0.701754


## 3. Per RX statistics
- max qty
- mean qty
- quartiles (25%, 50%, 75%)
- max qty/days_supply
- mean qty/days_supply

In [16]:
df_per_rx = df_final.groupby(['gcsn'])['qty','qty_per_day'].describe().unstack(-1)
to_keep = [('qty','mean'),('qty','25%'),('qty','50%'),('qty','75%'),('qty','max'),('qty_per_day','mean'),('qty_per_day','max')]
df_per_rx = df_per_rx[to_keep].fillna(0)
df_per_rx.columns.set_levels(levels=['units_per_rx','rx_units_per_day'], level=0,inplace=True)
df_per_rx.head()

units_per_rx                             rx_units_per_day     
              mean    25%    50%    75%    max             mean  max
gcsn                                                                
266.0    70.000000   60.0   90.0   90.0   90.0         1.000000  1.0
346.0   180.000000  180.0  180.0  180.0  180.0         2.000000  2.0
390.0    71.111111   51.0   90.0   90.0  200.0         1.013889  2.0
391.0    80.289855   58.0   90.0   90.0  180.0         1.116264  2.0
392.0    73.268657   60.0   90.0   90.0  150.0         1.000000  1.0

## 4. Per CLINIC statistics

- Number of Rx dispensed
- max qty
- mean qty
- quartiles (25%, 50%, 75%)

In [17]:
df_pc_cnt = df_final.groupby(['gcsn','date'])['qty'].count().unstack(-1).T.describe().T[['mean','max']]
df_pc_cnt.columns = ['rx_cnt_mean','rx_cnt_max']
df_pc_cnt.head()

,rx_cnt_mean,rx_cnt_max
gcsn,,
266.0,1.000000,1.0
346.0,1.000000,1.0
390.0,1.285714,3.0
391.0,1.815789,4.0
392.0,1.595238,5.0


In [18]:
df_pc_rx = df_final.groupby(['gcsn','date'])['qty'].sum().unstack(-1).T.describe().T
df_pc_rx.head()

,count,mean,std,min,25%,50%,75%,max
gcsn,,,,,,,,
266.0,3.0,70.000000,34.641016,30.0,60.0,90.0,90.0,90.0
346.0,2.0,180.000000,0.000000,180.0,180.0,180.0,180.0,180.0
390.0,28.0,91.428571,55.407867,7.0,60.0,90.0,92.5,260.0
391.0,38.0,145.789474,80.642406,7.0,90.0,120.5,202.5,300.0
392.0,42.0,116.880952,68.965245,7.0,90.0,90.0,172.5,280.0


In [19]:
df_per_clinic = df_pc_rx.join(df_pc_cnt).fillna(0)
df_per_clinic.columns = pd.MultiIndex.from_tuples([('per_clinic',col) for col in df_per_clinic.columns])
df_per_clinic.head()

per_clinic                                                            \
           count        mean        std    min    25%    50%    75%    max   
gcsn                                                                         
266.0        3.0   70.000000  34.641016   30.0   60.0   90.0   90.0   90.0   
346.0        2.0  180.000000   0.000000  180.0  180.0  180.0  180.0  180.0   
390.0       28.0   91.428571  55.407867    7.0   60.0   90.0   92.5  260.0   
391.0       38.0  145.789474  80.642406    7.0   90.0  120.5  202.5  300.0   
392.0       42.0  116.880952  68.965245    7.0   90.0   90.0  172.5  280.0   

                              
      rx_cnt_mean rx_cnt_max  
gcsn                          
266.0    1.000000        1.0  
346.0    1.000000        1.0  
390.0    1.285714        3.0  
391.0    1.815789        4.0  
392.0    1.595238        5.0

In [20]:
df_rx_info = df_ppc
df_rx_info = df_rx_info.join(df_gcsn_agg,how='left')
df_rx_info = df_rx_info.join(df_per_rx,how='left')
df_rx_info = df_rx_info.join(df_per_clinic,how='left')
df_rx_info.fillna(0,inplace=True)
df_rx_info.head()

pp_info                                                 \
                ndc                            desc pkg_size units   
gcsn                                                                 
266.0  1.850144e+08  AMIODARONE HCL 200 MG TABLESAN      500    TA   
287.0  5.011103e+10    HYDRALAZINE 50 MG TABLET PLI      100    TA   
346.0  2.282127e+08  CLONIDINE HCL 0.1 MG TABLETACT      100    TA   
390.0  1.850610e+08     LISINOPRIL 10 MG TABLET SAN     1000    TA   
391.0  1.850620e+08     LISINOPRIL 20 MG TABLET SAN     1000    TA   

                                                    gcsn_totals            \
      reorder_point reorder_qty reorder_qoh_desired    rx_count tot_units   
gcsn                                                                        
266.0         300.0         0.0               300.0         3.0     210.0   
287.0         200.0         0.0               200.0         0.0       0.0   
346.0         270.0         0.0               270.0         2.0     360.0   
390.0         427.0         0.0              3000.0        36.0    2560.0   
391.0         643.0         0.0              3000.0        69.0    5540.0   

                 ...     per_clinic                                       \
          qoh    ...          count        mean        std    min    25%   
gcsn             ...                                                       
266.0   589.0    ...            3.0   70.000000  34.641016   30.0   60.0   
287.0   500.0    ...            0.0    0.000000   0.000000    0.0    0.0   
346.0   360.0    ...            2.0  180.000000   0.000000  180.0  180.0   
390.0  2265.0    ...           28.0   91.428571  55.407867    7.0   60.0   
391.0  2730.0    ...           38.0  145.789474  80.642406    7.0   90.0   

                                                   
         50%    75%    max rx_cnt_mean rx_cnt_max  
gcsn                                               
266.0   90.0   90.0   90.0    1.000000        1.0  
287.0    0.0    0.0    0.0    0.000000        0.0  
346.0  180.0  180.0  180.0    1.000000        1.0  
390.0   90.0   92.5  260.0    1.285714        3.0  
391.0  120.5  202.5  300.0    1.815789        4.0  

[5 rows x 27 columns]

In [21]:
df_rx_info.to_excel('rx_info.xlsx')